<a href="https://colab.research.google.com/github/Tyler9937/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/Copy_of_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
# Loading in local file
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [0]:
# Importing pandas
import pandas as pd
df = pd.read_csv('dataset.csv')

In [0]:
# Checking shape
df.shape

(7043, 21)

In [0]:
# Checking head
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
# Dropping uneeded columns
df = df.drop(columns=['customerID'])

In [0]:
# Creating func for changing y var
def binary(x):
  if x == 'Yes':
    return 1
  else:
    return 0

In [0]:
# Appling binary func
df['Churn'] = df['Churn'].apply(binary)

In [0]:
# Checking for sucsess
df['Churn']

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [0]:
# Checking datatypes
df['MonthlyCharges'].dtype

dtype('float64')

In [0]:
# Importing numpy and creating func to change messy strings to floats
import numpy as np

def to_float(x):
  try:
    float(x)
  except:
    x = np.nan
  return float(x)

In [0]:
# applying to float func
df['TotalCharges'] = df['TotalCharges'].apply(to_float)

In [0]:
# Looking at nan values
df['TotalCharges'].isna().sum()

11

In [0]:
# Looking at all uniqe values 
for col in df.columns:
  print(df[col].unique())

['Female' 'Male']
[0 1]
['Yes' 'No']
['No' 'Yes']
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
['No' 'Yes']
['No phone service' 'No' 'Yes']
['DSL' 'Fiber optic' 'No']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['Month-to-month' 'One year' 'Two year']
['Yes' 'No']
['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
[0 1]


In [0]:
# Checking for nan values
for col in df.columns:
  print(df[col].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11
0


In [0]:
# Replacing nan values with means of columns
df = df.fillna(df.mean())

In [0]:
# Checking for sucsess
for col in df.columns:
  print(df[col].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [0]:
# Creating feature and target values
X = df.drop(columns=['Churn'])
y = df['Churn']

In [0]:
# Importin needed librares and preforming train test split
import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
              X, y, test_size=0.33, random_state=42)

In [0]:
# proferming one hot encoding on data
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [0]:
# Normalizing the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_encoded = scaler.fit_transform(X_train_encoded)
X_test_encoded = scaler.transform(X_test_encoded)
print(X_train_encoded[0])

In [0]:
# Creating the keras model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = X_train_encoded.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile Model
model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics=['accuracy'])

# Fit Model
model.fit(X_train_encoded, y_train, 
          validation_data=(X_test_encoded,y_test), 
          epochs=epochs, 
          batch_size=batch_size,
          verbose=0
         )

In [0]:
model.predict(X_train_encoded)

array([[ 0.81004465],
       [ 0.9796419 ],
       [-4.296499  ],
       ...,
       [-2.9292045 ],
       [ 0.88807744],
       [-5.1833262 ]], dtype=float32)

Trying to implement gridsearch example used in class

In [0]:
X_train_encoded.shape

(4718, 45)

In [0]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# split into input (X) and output (Y) variables
X = X_train_encoded
Y = y_train

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7986459016799927 using {'batch_size': 20, 'epochs': 20}
Means: 0.7903798341751098, Stdev: 0.010185291382498997 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7986459016799927, Stdev: 0.005952399210878483 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7886804103851318, Stdev: 0.010114466924459687 with: {'batch_size': 40, 'epochs': 20}
Means: 0.791440737247467, Stdev: 0.008609846353613513 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7965247988700866, Stdev: 0.014493507009780091 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7980096459388732, Stdev: 0.013066237160359352 with: {'batch_size': 100, 'epochs': 20}


In [0]:
# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20, 40, 60, 200]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7939826488494873 using {'batch_size': 20, 'epochs': 40}
Means: 0.7899561047554016, Stdev: 0.00792825916057379 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7939826488494873, Stdev: 0.010802716214485768 with: {'batch_size': 20, 'epochs': 40}
Means: 0.7929222106933593, Stdev: 0.005616387012401965 with: {'batch_size': 20, 'epochs': 60}
Means: 0.781053078174591, Stdev: 0.010231065166315391 with: {'batch_size': 20, 'epochs': 200}


In [0]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# split into input (X) and output (Y) variables
X = X_train_encoded
Y = y_train

# Function to create model, required for KerasClassifier
def create_model(optimizer = 'adam', nodes= 45):
    # create model
    model = Sequential()
    model.add(Dense(45, input_dim=45, activation='relu'))
    model.add(Dense(nodes, activation='relu'))
    model.add(Dense(nodes, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [40],
              'nodes': [10, 45, 90, 180]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7626134634017945 using {'batch_size': 20, 'epochs': 40, 'nodes': 10}
Means: 0.7626134634017945, Stdev: 0.011738252305378298 with: {'batch_size': 20, 'epochs': 40, 'nodes': 10}
Means: 0.7492642045021057, Stdev: 0.017192453820506447 with: {'batch_size': 20, 'epochs': 40, 'nodes': 45}
Means: 0.7359014749526978, Stdev: 0.008678692556408674 with: {'batch_size': 20, 'epochs': 40, 'nodes': 90}
Means: 0.740570342540741, Stdev: 0.005048090714050068 with: {'batch_size': 20, 'epochs': 40, 'nodes': 180}


In [0]:
# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [40],
              'nodes': [],
              'optimizer': ['adam', 'SGD', 'RMSprop', 'Adamax']'}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?